In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import linregress
from itertools import product
import math

pd.options.plotting.backend = "plotly"

In [ ]:
# https://docs.screeps.com/api/#Creep
CREEP_TICKS_LIFE = 1500
MAX_BODY_SIZE = 50
MAX_DISTANCE = 50

In [ ]:
# https://docs.screeps.com/creeps.html#Movement'

class FixedWalk:
    """Assuming one moving part for each carry part"""

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def round_trip(cls, work, carry, distance):
        return (distance * 2) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry):
        move = carry + work
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def profit(cls, work, carry, distance):
        return round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance) * cls.capacity(carry) - cls.body_cost(work, carry))

    @classmethod
    def cost_benefit(cls, work, carry, distance):
        return cls.profit(work, carry, distance) / cls.body_cost(work, carry)

    @classmethod
    def cost_benefit_fixed_walk(cls):
        actual_max_size = round(MAX_BODY_SIZE / 4)
        work_parts = [*range(1, actual_max_size)]
        carry_parts = [*range(1, actual_max_size)]
        distances = [*range(5, MAX_DISTANCE, 5)]

        cost_benefit_heatmap = list(map(lambda wc: [str(wc), wc[0], wc[1], wc[2], cls.cost_benefit(*wc)], product(work_parts, carry_parts, distances)))
        df = pd.DataFrame(cost_benefit_heatmap, columns=["body","work_parts", "carry_parts", "distance", "cost_benefit"])
        df["ratio"] = df['carry_parts'] / df['work_parts']
        df = df.sort_values(by="cost_benefit")
        df['ratio'] = round(df['ratio'], 4)
        df['cost_benefit'] = round(df['cost_benefit'], 4)

        best_by_distance = df.groupby(['distance']).agg({'cost_benefit': 'max'})
        print(best_by_distance)
        print("")

        df_reg = df[df['cost_benefit'].isin(best_by_distance['cost_benefit'])]
        df_reg = df_reg.sort_values('distance').copy()
        print(df_reg)
        print("")

        regression = linregress(df_reg['distance'], df_reg['ratio'])
        print(regression)
        print("")

        print(round(regression.slope, 3), round(regression.intercept, 3))
        print("")

        return px.scatter(df, x="ratio", y="cost_benefit", color="distance")

FixedWalk.cost_benefit_fixed_walk()

In [ ]:
class Proportions:
    """Not assuming one moving part for each carry part"""

    @classmethod
    def harvested_tick(cls, work):
        return work * 2

    @classmethod
    def capacity(cls, carry):
        return carry * 50

    @classmethod
    def ticks_to_full(cls, work, carry):
        return cls.capacity(carry) / cls.harvested_tick(work)

    @classmethod
    def ticks_per_move(work, carry, move, direction="Forward"):
        return math.floor(work / move) if direction=="Forward" else math.floor((work + carry) / move)

    @classmethod
    def round_trip(cls, work, carry, move, distance):
        forward = distance * cls.ticks_per_move(work, carry, move, "Forward") + distance
        backwards = distance * cls.ticks_per_move(work, carry, move, "Backwards") + distance
        return (forward + backwards) + cls.ticks_to_full(work, carry) + 1

    @classmethod
    def body_cost(cls, work, carry, move):
        return work * 100 + carry * 50 + move * 50

    @classmethod
    def profit(cls, work, carry, move, distance):
        round_trips = round(CREEP_TICKS_LIFE / cls.round_trip(work, carry, distance, move))
        return round_trips * cls.capacity(carry) - cls.body_cost(work, carry, move)

    @classmethod
    def cost_benefit(cls, work, carry, move, distance):
        return cls.profit(work, carry, distance, move) / cls.body_cost(work, carry, move)
    
    @classmethod
    def cost_benefit_regression(cls):
        pass









Proportions.cost_benefit_regression()

In [ ]:
actual_max_size = round(MAX_BODY_SIZE / 4)
work_parts = [*range(1, actual_max_size)]
carry_parts = [*range(1, actual_max_size)]
distances = [*range(5, MAX_DISTANCE, 5)]